# 文字認識モデルの学習


In [1]:
import pandas as pd
from PIL import Image
import torch
import torchvision
from torchvision import transforms as transforms
import numpy as np
from torch.utils.data import TensorDataset,DataLoader

# 全結合層と活性化関数
from torch import nn
from torch.nn import functional as F

# 損失関数と最適化関数
from torch import optim

import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\tonbe\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 指定されたプロシージャが見つかりません。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


/pictures にあるinformation.csvから画像名と画像のラベルのセットを読み込む

In [2]:
df = pd.read_csv("pictures/information.csv")
df = df[["filename","label"]]
filelist=df.values.tolist()


画像の前処理を行う
この際にオーグメンテーションをして学習データを増やす

In [4]:
#リストの初期化
images = []
labels = []


#リサイズ後の画像のサイズを決めておく
common_size = (28, 28)

#オーグメンテーションの手順 
transform_augment = transforms.Compose([
    transforms.Resize(common_size),
    #transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Grayscale(),
    transforms.ToTensor(),
])

# グレースケール化
transform_normal = transforms.Compose([
    transforms.Resize(common_size),
    transforms.Grayscale(),
    transforms.ToTensor(),
])

# 画像の読み込み
for filename, label in filelist:
    if filename[-4:] == ".png":
        p = Image.open('./pictures/' + filename)
    else:
        p = Image.open('./pictures/' + filename + ".png")
    
    # オーギュメンテーションをしてリストに追加
    for i in range(3):
        p_augmented = transform_augment(p)
        images.append(p_augmented)
        labels.append(torch.tensor(label, dtype=torch.int64))


    
    #元画像もリストに追加
    p_gray = transform_normal(p)
    images.append(p_gray)
    labels.append(torch.tensor(label, dtype=torch.int64))

images = torch.stack(images, dim=0)
labels = torch.stack(labels, dim=0)

# データセットの作成
dataset = TensorDataset(images, labels)

# 学習データテストデータに分ける
r = 0.9
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(r*len(dataset)), len(dataset) - int(r*len(dataset))])

#データローダーの準備
batch_size = 27
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [5]:
train_dataset[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.1961, 0.5020, 0.5020, 0.5020, 0.5020, 0.4235,
           0.0000, 0.0000, 0.0000, 0.0000, 

認識モデルの定義

In [6]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output

モデルの学習を行う

In [7]:
from tqdm import tqdm

model = MNISTModel()

criterion = nn.NLLLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(100):
    total_loss = 0
    for images, labels in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    # loss は平均を取って表示する
    print(f'Epoch: {epoch + 1}, Loss: {total_loss / len(train_loader)}')

# Torch Script形式で保存
#model_scripted = torch.jit.script(model)
#model_scripted.save('DSST_model/DSST_model.pth')
torch.save(model.state_dict(), 'letter_recognition_model.pth')

100%|██████████| 5/5 [00:00<00:00, 73.04it/s]


Epoch: 1, Loss: 2.302913951873779


100%|██████████| 5/5 [00:00<00:00, 100.17it/s]


Epoch: 2, Loss: 2.2963420867919924


100%|██████████| 5/5 [00:00<00:00, 104.08it/s]


...

Epoch: 98, Loss: 0.03810674846172333


100%|██████████| 5/5 [00:00<00:00, 99.44it/s]


Epoch: 99, Loss: 0.02875652089715004


100%|██████████| 5/5 [00:00<00:00, 99.52it/s]

Epoch: 100, Loss: 0.021084033139050006


モデルの精度を評価する

In [8]:
correct = 0
total = 0

# Torch Script形式で読み込み
test_model = MNISTModel()
test_model.load_state_dict(torch.load('letter_recognition_model.pth', map_location=device))
test_model.eval()
#model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")


Accuracy: 100.0%
